# Data Cleaning

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.linear_model import perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [25]:
excel = pd.ExcelFile('HainanClean_New.xlsx')
hainan = excel.parse("fulldf")
hainan
hainan.columns = hainan.columns.str.replace('  ', '_')
hainan.columns = hainan.columns.str.replace(' ', '_')
hainan.columns = hainan.columns.str.replace('(', '')
hainan.columns = hainan.columns.str.replace('（', '')
hainan.columns = hainan.columns.str.replace(')', '')

d = {'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6,
     'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}
hainan.Month = hainan.Month.map(d)
hainan.BioCNG_Produced_Nm3 = hainan.BioCNG_Produced_m3.shift(-15)
hainan.drop(hainan.tail(15).index,inplace=True)
hainan = hainan[np.isfinite(hainan['Month'])]
hainan['BioCNG_cumsum'] = hainan.BioCNG_Produced_m3.cumsum()
hainan.columns
hainan = hainan[np.isfinite(hainan['Lemon_waste_t'])]
hainan = hainan[np.isfinite(hainan['Percolate_t'])]
hainan = hainan.replace(' ',0)
hainan = hainan.replace('',0)
hainan = hainan.replace('  ',0)
hainan = hainan.drop(['Year', 'Month', 'Day', 'Month_#', 'Day_#', 'Raw_Biogas_Produced_m3', 'BioCNG_Sold_m3', 'Vehicle_use_m3',
       'Liquid_Fertilizer_Produced_t', 'Solid_fertilizer_produced_t',
       'Wastewater_flow_to_WWTP_unit?', 'Solid_residues_kg','50%_NaOH/kg', 'FeCl2/kg', 'PAM/kg',
       'Defoamer/kg', 'Project_electricity_use/kWh',
       'Office_space_electricity_use/kWh', 'Water/m3', 'Diesel/L'], axis=1)
sumcum = hainan['BioCNG_cumsum']
hainan.drop(labels=['BioCNG_cumsum'], axis=1,inplace = True)
hainan.insert(0, 'BioCNG_cumsum', sumcum)
hainan.head()
hainan = hainan.fillna(0);
hainan = hainan.reset_index(drop=True)
hainan.shape

(1373, 17)

In [26]:
#Moving Average

hainan['BioCNG_Produced_m3']
hainan['MA'] = hainan['BioCNG_Produced_m3'].rolling(window=1).mean()
hainan['MA'][5] = 0
for i in range(5):
    hainan['MA'][5] += hainan['MA'][i]
hainan['MA'][5] = hainan['MA'][5]/5
for i in range(6,1373):
    hainan['MA'][i] = (hainan['MA'][i-1]*5 
                        - hainan['BioCNG_Produced_m3'][i-6] 
                        + hainan['BioCNG_Produced_m3'][i-1])/5
hainan = hainan.fillna(0);
hainan['MA'];

# Preliminary Work

In [27]:
hainan_train, hainan_test = train_test_split(hainan, test_size=0.2)
print('train data len:',len(hainan_train))
print('test data len:',len(hainan_test))

train data len: 1098
test data len: 275


In [28]:
#ols
hainan_ols = ols("BioCNG_cumsum ~ Pig_Manure_t + Cassava_t + Fish_waste_water_t +\
                    Kitchen_food_waste_t + Municipal_fecal_residue_t + Tea_waste_t +\
                    Chicken_litter_t + Bagasse_feed_t + Alcohol_waste_t +\
                    Chinese_medicine_waste_t + Energy_grass_t + Banana_fruit_shafts_t +\
                    Lemon_waste_t + Percolate_t + Other_waste_t", data=hainan_train).fit()
hainan_ols_summary = hainan_ols.summary()
hainan_ols_summary


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          BioCNG_cumsum   R-squared:                       0.843
Model:                            OLS   Adj. R-squared:                  0.841
Method:                 Least Squares   F-statistic:                     388.2
Date:                Fri, 23 Mar 2018   Prob (F-statistic):               0.00
Time:                        17:17:49   Log-Likelihood:                -16455.
No. Observations:                1098   AIC:                         3.294e+04
Df Residuals:                    1082   BIC:                         3.302e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                  1.389e+06   4.93e+04     28.190      0.000    1.29e+06    1.49e+06
Pig_Manure_t               3.407e+04   2575.049     13.230      0.000     2.9e+04    3.91e+04
Cassava_t                  -475.3583   1032.277     -0.460      0.645   -2500.850    1550.134
Fish_waste_water_t         2346.3520   4841.329      0.485      0.628   -7153.104    1.18e+04
Kitchen_food_waste_t       2.935e+04   2440.792     12.026      0.000    2.46e+04    3.41e+04
Municipal_fecal_residue_t  5258.9560    966.312      5.442      0.000    3362.899    7155.013
Tea_waste_t                3.135e+04   8744.805      3.585      0.000    1.42e+04    4.85e+04
Chicken_litter_t           1.603e+04   4119.197      3.891      0.000    7945.767    2.41e+04
Bagasse_feed_t            -1.587e+04   1654.532     -9.592      0.000   -1.91e+04   -1.26e+04
Alcohol_waste_t            3.484e+04   6795.662      5.127      0.000    2.15e+04    4.82e+04
Chinese_medicine_waste_t   2.736e+04   2.25e+04      1.215      0.224   -1.68e+04    7.15e+04
Energy_grass_t             2.883e+04   1.77e+04      1.629      0.104   -5888.569    6.35e+04
Banana_fruit_shafts_t      1607.3763   6953.034      0.231      0.817    -1.2e+04    1.53e+04
Lemon_waste_t             -1867.6685   2.03e+04     -0.092      0.927   -4.17e+04    3.79e+04
Percolate_t                2461.1317    291.711      8.437      0.000    1888.748    3033.515
Other_waste_t              8059.1205   2246.195      3.588      0.000    3651.729    1.25e+04
==============================================================================
Omnibus:                       35.930   Durbin-Watson:                   2.040
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               24.110
Skew:                           0.241   Prob(JB):                     5.82e-06
Kurtosis:                       2.457   Cond. No.                         348.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [29]:
#ols
hainan_ols = ols("BioCNG_Produced_m3 ~ MA", data=hainan_train).fit()
hainan_ols_summary = hainan_ols.summary()
hainan_ols_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     BioCNG_Produced_m3   R-squared:                       0.842
Model:                            OLS   Adj. R-squared:                  0.842
Method:                 Least Squares   F-statistic:                     5842.
Date:                Fri, 23 Mar 2018   Prob (F-statistic):               0.00
Time:                        17:17:49   Log-Likelihood:                -9595.8
No. Observations:                1098   AIC:                         1.920e+04
Df Residuals:                    1096   BIC:                         1.921e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    221.5836     80.288      2.760      0.006      64.048     379.120
MA             0.9559      0.013     76.434      0.000       0.931       0.980
==============================================================================
Omnibus:                      360.695   Durbin-Watson:                   1.952
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4437.498
Skew:                          -1.150   Prob(JB):                         0.00
Kurtosis:                      12.576   Cond. No.                     1.13e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.13e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#ols
hainan_ols = ols("Solid_fertilizer_produced_t ~ Pig_Manure_t + Cassava_t + Fish_waste_water_t +\
                    Kitchen_food_waste_t + Municipal_fecal_residue_t + Tea_waste_t +\
                    Chicken_litter_t + Bagasse_feed_t + Alcohol_waste_t +\
                    Chinese_medicine_waste_t + Energy_grass_t + Banana_fruit_shafts_t +\
                    Lemon_waste_t + Percolate_t + Other_waste_t", data=hainan_train).fit()
hainan_ols_summary = hainan_ols.summary()
hainan_ols_summary

From the ols results based on three different dependent variable, we can see that only the first one (cumulative sum of BioCNG) has a relatively high value for R-squared. So We decided to focus on the first one.

# Moving Average

In [30]:
# Predict on the training data
X_train = hainan_train[['MA']]
y_train = hainan_train.BioCNG_Produced_m3
y_pred_train = hainan_ols.predict(X_train)

# Compute the root-mean-square of training data
rms_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
print('Training error:',rms_train/sum(y_train))

# Predict on the test data
X_test = hainan_test[['MA']]
y_test = hainan_test.BioCNG_Produced_m3
y_pred_test = hainan_ols.predict(X_test)

# Compute the root-mean-square of test data
rms_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
print('Testing error:',rms_test/sum(y_test))

hainan_train

Training error: 0.00026106074171
Testing error: 0.000989094515775


,BioCNG_cumsum,BioCNG_Produced_m3,Pig_Manure_t,Cassava_t,Fish_waste_water_t,Kitchen_food_waste_t,Municipal_fecal_residue_t,Tea_waste_t,Chicken_litter_t,Bagasse_feed_t,Alcohol_waste_t,Chinese_medicine_waste_t,Energy_grass_t,Banana_fruit_shafts_t,Lemon_waste_t,Percolate_t,Other_waste_t,MA
241,569862.00,5582.0,8.50,40.00,0.00,0.00,0.00,0.00,0.00,8.20,0.00,0.00,0.0,0.00,0.00,0.0,0.00,3771.800
1354,7205722.47,11936.0,44.60,0.00,0.00,120.00,115.34,0.00,35.10,6.30,0.00,0.00,0.0,0.00,6.68,280.0,0.00,11909.800
883,2982776.00,1914.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,2743.800
494,1591323.00,1770.0,11.90,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,2061.200
546,1659132.00,1341.0,5.96,0.00,0.00,0.00,0.00,0.00,0.00,6.30,0.00,0.00,0.0,0.00,0.00,0.0,0.00,1478.400
467,1534980.00,2163.0,8.12,0.00,13.34,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,2532.600
1052,4007382.00,10721.0,27.00,0.00,0.00,14.70,31.04,0.00,13.22,0.00,0.00,5.58,0.0,0.00,0.00,262.0,0.00,9024.200
496,1594558.00,1681.0,17.10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,1880.800
125,296369.00,2184.0,3.18,0.00,0.00,0.00,0.00,0.00,0.00,20.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,1943.000
699,2394524.00,3410.0,14.00,0.00,7.62,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,6236.400


In [31]:
#logistic regression
hainan['BioCNG_Produced_m3'] = pd.cut(hainan['BioCNG_Produced_m3'], bins=3, labels=[0, 1, 2])
hainan_train, hainan_test = train_test_split(hainan, test_size=0.2)
y_train = hainan_train.BioCNG_Produced_m3
y_test = hainan_test.BioCNG_Produced_m3
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_test = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(logreg.score(X_train, y_train)))
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred_test)
print(confusion_matrix)

Accuracy of logistic regression classifier on train set: 0.58
Accuracy of logistic regression classifier on test set: 0.65
[[180   0   0]
 [ 51   0   0]
 [ 44   0   0]]


In [32]:
# SVM
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)  
y_pred_test = clf.predict(X_test)
print('Accuracy of SVM on train set: {:.2f}'.format(clf.score(X_train, y_train)))
print('Accuracy of SVM on test set: {:.2f}'.format(clf.score(X_test, y_test)))

Accuracy of SVM on train set: 0.95
Accuracy of SVM on test set: 0.63


In [33]:
# kNN
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=4)
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(hainan[['BioCNG_Produced_m3','MA']])
hainan_normalized = pd.DataFrame(np_scaled)
hainan_normalized_train, hainan_normalized_test = train_test_split(hainan_normalized, test_size=0.2)
Xnor_train = hainan_normalized_train[1].reshape(-1,1)
ynor_train = hainan_normalized_train[0]
Xnor_test = hainan_normalized_test[1].reshape(-1,1)
ynor_test = hainan_normalized_test[0]
knn.fit(Xnor_train, ynor_train)
y_pred_test = knn.predict(Xnor_test)
print('Accuracy of kNN on train set: {:.2f}'.format(knn.score(Xnor_train, ynor_train)))
print('Accuracy of kNN on test set: {:.2f}'.format(knn.score(Xnor_test, ynor_test)))

Accuracy of kNN on train set: 0.84
Accuracy of kNN on test set: 0.74


/Users/anna/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':
/Users/anna/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  # This is added back by InteractiveShellApp.init_path()


In [34]:
# xgboost
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
y_pred_test = xgb.predict(X_test)
print('Accuracy of XGboost on train set: {:.2f}'.format(xgb.score(X_train, y_train)))
print('Accuracy of XGboost on test set: {:.2f}'.format(xgb.score(X_test, y_test)))
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred_test)
print(confusion_matrix)

Accuracy of XGboost on train set: 0.61
Accuracy of XGboost on test set: 0.63
[[173   5   2]
 [ 50   1   0]
 [ 44   0   0]]


In [35]:
# random forest
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators = 1000)
random_forest.fit(X_train, y_train)
print('Accuracy of Random Forest on train set: {:.2f}'.format(random_forest.score(X_train, y_train)))
print('Accuracy of Random Forest on test set: {:.2f}'.format(random_forest.score(X_test, y_test)))

Accuracy of Random Forest on train set: 0.99
Accuracy of Random Forest on test set: 0.45


# Training Algorithms

In [36]:
# Predict on the training data
X_train = hainan_train[['Pig_Manure_t',
       'Cassava_t', 'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t', 'MA']]
y_train = hainan_train.BioCNG_cumsum
y_pred_train = hainan_ols.predict(X_train)

# Compute the root-mean-square of training data
rms_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
print('Training error:',rms_train/sum(y_train))

# Predict on the test data
X_test = hainan_test[['Pig_Manure_t',
       'Cassava_t', 'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t', 'MA']]
y_test = hainan_test.BioCNG_cumsum
y_pred_test = hainan_ols.predict(X_test)

# Compute the root-mean-square of test data
rms_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
print('Testing error:',rms_test/sum(y_test))

hainan_train.head()

Training error: 0.0011447310155
Testing error: 0.00444987967746


,BioCNG_cumsum,BioCNG_Produced_m3,Pig_Manure_t,Cassava_t,Fish_waste_water_t,Kitchen_food_waste_t,Municipal_fecal_residue_t,Tea_waste_t,Chicken_litter_t,Bagasse_feed_t,Alcohol_waste_t,Chinese_medicine_waste_t,Energy_grass_t,Banana_fruit_shafts_t,Lemon_waste_t,Percolate_t,Other_waste_t,MA
936,3087283.0,0,0.00,0.0,0.0,12.50,26.12,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1943.4
935,3087283.0,0,0.00,0.0,0.0,0.00,25.78,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,2450.6
801,2774998.0,0,6.26,0.0,0.0,0.00,0.00,0.00,0.0,3.68,4.74,0.0,0.0,0.0,0.0,0.0,0.0,686.0
1065,4125520.0,1,27.72,0.0,0.0,31.14,60.44,8.28,0.0,0.00,0.00,0.0,0.0,0.0,0.0,272.0,0.0,8494.8
112,263194.0,0,5.44,0.0,0.0,0.00,0.00,0.00,0.0,30.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,2575.0


1. Logistic regression
2. SVM
3. Perceptron
4. kNN
5. Random Forest
6. xgBoost

In [37]:
#logistic regression
hainan['BioCNG_cumsum'] = pd.cut(hainan['BioCNG_cumsum'], bins=3, labels=[0, 1, 2])
hainan_train, hainan_test = train_test_split(hainan, test_size=0.2)
y_train = hainan_train.BioCNG_cumsum
y_test = hainan_test.BioCNG_cumsum
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_test = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(logreg.score(X_train, y_train)))
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred_test)
print(confusion_matrix)

Accuracy of logistic regression classifier on train set: 0.52
Accuracy of logistic regression classifier on test set: 0.49
[[133   8   0]
 [ 87   2   0]
 [ 43   2   0]]


In [38]:
# SVM
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)  
y_pred_test = clf.predict(X_test)
print('Accuracy of SVM on train set: {:.2f}'.format(clf.score(X_train, y_train)))
print('Accuracy of SVM on test set: {:.2f}'.format(clf.score(X_test, y_test)))

Accuracy of SVM on train set: 1.00
Accuracy of SVM on test set: 0.51


In [39]:
# kNN
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=4)
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(hainan[['BioCNG_cumsum', 'Pig_Manure_t',
       'Cassava_t', 'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t','MA']])
hainan_normalized = pd.DataFrame(np_scaled)
hainan_normalized_train, hainan_normalized_test = train_test_split(hainan_normalized, test_size=0.2)
hainan_normalized
Xnor_train = hainan_normalized_train.iloc[:, 1:17]    
ynor_train = hainan_normalized_train[0]
Xnor_test = hainan_normalized_test.iloc[:, 1:17]
ynor_test = hainan_normalized_test[0]
knn.fit(Xnor_train, ynor_train)
y_pred_test = knn.predict(Xnor_test)
print('Accuracy of kNN on train set: {:.2f}'.format(knn.score(Xnor_train, ynor_train)))
print('Accuracy of kNN on test set: {:.2f}'.format(knn.score(Xnor_test, ynor_test)))


Accuracy of kNN on train set: 0.91
Accuracy of kNN on test set: 0.79


In [40]:
# xgboost
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
y_pred_test = xgb.predict(X_test)
print('Accuracy of XGboost on train set: {:.2f}'.format(xgb.score(X_train, y_train)))
print('Accuracy of XGboost on test set: {:.2f}'.format(xgb.score(X_test, y_test)))
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred_test)
print(confusion_matrix)

Accuracy of XGboost on train set: 0.66
Accuracy of XGboost on test set: 0.48
[[121  17   3]
 [ 78  10   1]
 [ 37   8   0]]


In [41]:
# random forest
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators = 1000)
random_forest.fit(X_train, y_train)
print('Accuracy of Random Forest on train set: {:.2f}'.format(random_forest.score(X_train, y_train)))
print('Accuracy of Random Forest on test set: {:.2f}'.format(random_forest.score(X_test, y_test)))

Accuracy of Random Forest on train set: 1.00
Accuracy of Random Forest on test set: 0.44


From the accuracy results of the above algorithms training, we can see that except for kNN, all of them have relatively good results for accuracy around 0.5.

# Trying with 1/x

In [21]:
for col in hainan.columns[2:]:
    hainan['1/'+col] = 1/(hainan[col])
hainan.head()
hainan.columns

Index(['BioCNG_cumsum', 'BioCNG_Produced_m3', 'Pig_Manure_t', 'Cassava_t',
       'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t', 'MA', '1/Pig_Manure_t', '1/Cassava_t',
       '1/Fish_waste_water_t', '1/Kitchen_food_waste_t',
       '1/Municipal_fecal_residue_t', '1/Tea_waste_t', '1/Chicken_litter_t',
       '1/Bagasse_feed_t', '1/Alcohol_waste_t', '1/Chinese_medicine_waste_t',
       '1/Energy_grass_t', '1/Banana_fruit_shafts_t', '1/Lemon_waste_t',
       '1/Percolate_t', '1/Other_waste_t', '1/MA'],
      dtype='object')

In [22]:
hainan.replace(float('inf'), 0, inplace = True);
hainan_train, hainan_test = train_test_split(hainan, test_size=0.2)
print('train data len:',len(hainan_train))
print('test data len:',len(hainan_test))
hainan.head()

OverflowError: cannot convert float infinity to integer

In [ ]:
# Predict on the training data
X_train = hainan_train[['BioCNG_cumsum', 'BioCNG_Produced_m3', 'Pig_Manure_t', 'Cassava_t',
       'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t', '1/Pig_Manure_t', '1/Cassava_t',
       '1/Fish_waste_water_t', '1/Kitchen_food_waste_t',
       '1/Municipal_fecal_residue_t', '1/Tea_waste_t', '1/Chicken_litter_t',
       '1/Bagasse_feed_t', '1/Alcohol_waste_t', '1/Chinese_medicine_waste_t',
       '1/Energy_grass_t', '1/Banana_fruit_shafts_t', '1/Lemon_waste_t',
       '1/Percolate_t', '1/Other_waste_t']]
y_train = hainan_train.BioCNG_cumsum
y_pred_train = hainan_ols.predict(X_train)

# Compute the root-mean-square of training data
rms_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
print('Training error:',rms_train/sum(y_train))

# Predict on the test data
X_test = hainan_test[['BioCNG_cumsum', 'BioCNG_Produced_m3', 'Pig_Manure_t', 'Cassava_t',
       'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t', '1/Pig_Manure_t', '1/Cassava_t',
       '1/Fish_waste_water_t', '1/Kitchen_food_waste_t',
       '1/Municipal_fecal_residue_t', '1/Tea_waste_t', '1/Chicken_litter_t',
       '1/Bagasse_feed_t', '1/Alcohol_waste_t', '1/Chinese_medicine_waste_t',
       '1/Energy_grass_t', '1/Banana_fruit_shafts_t', '1/Lemon_waste_t',
       '1/Percolate_t', '1/Other_waste_t']]
y_test = hainan_test.BioCNG_cumsum
y_pred_test = hainan_ols.predict(X_test)

# Compute the root-mean-square of test data
rms_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
print('Testing error:',rms_test/sum(y_test))

hainan_train.head()

In [ ]:
#logistic regression
hainan['BioCNG_cumsum'] = pd.cut(hainan['BioCNG_cumsum'], bins=3, labels=[0, 1, 2])
hainan_train, hainan_test = train_test_split(hainan, test_size=0.2)
y_train = hainan_train.BioCNG_cumsum
y_test = hainan_test.BioCNG_cumsum
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_test = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(logreg.score(X_train, y_train)))
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred_test)
print(confusion_matrix)

In [ ]:
# SVM
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)  
y_pred_test = clf.predict(X_test)
print('Accuracy of SVM on train set: {:.2f}'.format(clf.score(X_train, y_train)))
print('Accuracy of SVM on test set: {:.2f}'.format(clf.score(X_test, y_test)))

In [ ]:
# perceptron
from sklearn.linear_model import perceptron
net = perceptron.Perceptron(n_iter=100, verbose=0, random_state=None, fit_intercept=True, eta0=0.002)
net.fit(X_train, y_train)
y_pred_test = net.predict(X_test)
print('Accuracy of perceptron on train set: {:.2f}'.format(net.score(X_train, y_train)))
print('Accuracy of perceptron on test set: {:.2f}'.format(net.score(X_test, y_test)))

In [ ]:
# kNN
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=4)
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(hainan[['BioCNG_cumsum', 'BioCNG_Produced_m3', 'Pig_Manure_t', 'Cassava_t',
       'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t', '1/Pig_Manure_t', '1/Cassava_t',
       '1/Fish_waste_water_t', '1/Kitchen_food_waste_t',
       '1/Municipal_fecal_residue_t', '1/Tea_waste_t', '1/Chicken_litter_t',
       '1/Bagasse_feed_t', '1/Alcohol_waste_t', '1/Chinese_medicine_waste_t',
       '1/Energy_grass_t', '1/Banana_fruit_shafts_t', '1/Lemon_waste_t',
       '1/Percolate_t', '1/Other_waste_t']])
hainan_normalized = pd.DataFrame(np_scaled)
hainan_normalized_train, hainan_normalized_test = train_test_split(hainan_normalized, test_size=0.2)
hainan_normalized
Xnor_train = hainan_normalized_train.iloc[:, 1:16]    
ynor_train = hainan_normalized_train[0]
Xnor_test = hainan_normalized_test.iloc[:, 1:16]
ynor_test = hainan_normalized_test[0]
knn.fit(Xnor_train, ynor_train)
y_pred_test = knn.predict(Xnor_test)
print('Accuracy of kNN on train set: {:.2f}'.format(knn.score(Xnor_train, ynor_train)))
print('Accuracy of kNN on test set: {:.2f}'.format(knn.score(Xnor_test, ynor_test)))


In [ ]:
# random forest
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators = 1000)
random_forest.fit(X_train, y_train)
print('Accuracy of Random Forest on train set: {:.2f}'.format(random_forest.score(X_train, y_train)))
print('Accuracy of Random Forest on test set: {:.2f}'.format(random_forest.score(X_test, y_test)))

# Trying with x^2

In [42]:
for col in hainan.columns[2:]:
    hainan[col+"**2"] = (hainan[col])**2
hainan.head()

,BioCNG_cumsum,BioCNG_Produced_m3,Pig_Manure_t,Cassava_t,Fish_waste_water_t,Kitchen_food_waste_t,Municipal_fecal_residue_t,Tea_waste_t,Chicken_litter_t,Bagasse_feed_t,...,Chicken_litter_t**2,Bagasse_feed_t**2,Alcohol_waste_t**2,Chinese_medicine_waste_t**2,Energy_grass_t**2,Banana_fruit_shafts_t**2,Lemon_waste_t**2,Percolate_t**2,Other_waste_t**2,MA**2
0,0,0,6.82,0.0,0.0,0.0,0.0,0.0,0.0,15.0,...,0.0,225.0,0.0,0.0,0.0,0.0,0.0,0.0,699.0736,4096576.0
1,0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,15.0,...,0.0,225.0,0.0,0.0,0.0,0.0,0.0,0.0,836.3664,8462281.0
2,0,0,6.66,0.0,0.0,0.0,0.0,0.0,0.0,15.0,...,0.0,225.0,0.0,0.0,0.0,0.0,0.0,0.0,723.6100,9120400.0
3,0,0,12.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,9253764.0
4,0,0,9.54,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,81.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,7722841.0


In [43]:
hainan_train, hainan_test = train_test_split(hainan, test_size=0.2)
print('train data len:',len(hainan_train))
print('test data len:',len(hainan_test))
hainan.head()
hainan.columns

train data len: 1098
test data len: 275


Index(['BioCNG_cumsum', 'BioCNG_Produced_m3', 'Pig_Manure_t', 'Cassava_t',
       'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t', 'MA', 'Pig_Manure_t**2', 'Cassava_t**2',
       'Fish_waste_water_t**2', 'Kitchen_food_waste_t**2',
       'Municipal_fecal_residue_t**2', 'Tea_waste_t**2', 'Chicken_litter_t**2',
       'Bagasse_feed_t**2', 'Alcohol_waste_t**2',
       'Chinese_medicine_waste_t**2', 'Energy_grass_t**2',
       'Banana_fruit_shafts_t**2', 'Lemon_waste_t**2', 'Percolate_t**2',
       'Other_waste_t**2', 'MA**2'],
      dtype='object')

In [51]:
# Predict on the training data
X_train = hainan_train[['BioCNG_cumsum', 'BioCNG_Produced_m3', 'Pig_Manure_t', 'Cassava_t',
       'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t', 'MA', 'Pig_Manure_t**2', 'Cassava_t**2',
       'Fish_waste_water_t**2', 'Kitchen_food_waste_t**2',
       'Municipal_fecal_residue_t**2', 'Tea_waste_t**2', 'Chicken_litter_t**2',
       'Bagasse_feed_t**2', 'Alcohol_waste_t**2',
       'Chinese_medicine_waste_t**2', 'Energy_grass_t**2',
       'Banana_fruit_shafts_t**2', 'Lemon_waste_t**2', 'Percolate_t**2',
       'Other_waste_t**2', 'MA**2']]
y_train = hainan_train.BioCNG_cumsum
y_pred_train = hainan_ols.predict(X_train)

# Predict on the test data
X_test = hainan_test[['BioCNG_cumsum', 'BioCNG_Produced_m3', 'Pig_Manure_t', 'Cassava_t',
       'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t', 'MA', 'Pig_Manure_t**2', 'Cassava_t**2',
       'Fish_waste_water_t**2', 'Kitchen_food_waste_t**2',
       'Municipal_fecal_residue_t**2', 'Tea_waste_t**2', 'Chicken_litter_t**2',
       'Bagasse_feed_t**2', 'Alcohol_waste_t**2',
       'Chinese_medicine_waste_t**2', 'Energy_grass_t**2',
       'Banana_fruit_shafts_t**2', 'Lemon_waste_t**2', 'Percolate_t**2',
       'Other_waste_t**2', 'MA**2']]
y_test = hainan_test.BioCNG_cumsum
y_pred_test = hainan_ols.predict(X_test)


In [52]:
#logistic regression
hainan['BioCNG_cumsum'] = pd.cut(hainan['BioCNG_cumsum'], bins=3, labels=[0, 1, 2])
hainan_train, hainan_test = train_test_split(hainan, test_size=0.2)
y_train = hainan_train.BioCNG_cumsum
y_test = hainan_test.BioCNG_cumsum
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_test = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(logreg.score(X_train, y_train)))
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred_test)
print(confusion_matrix)

Accuracy of logistic regression classifier on train set: 0.55
Accuracy of logistic regression classifier on test set: 0.45
[[124   1   0]
 [107   1   0]
 [ 42   0   0]]


In [53]:
# SVM
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)  
y_pred_test = clf.predict(X_test)
print('Accuracy of SVM on train set: {:.2f}'.format(clf.score(X_train, y_train)))
print('Accuracy of SVM on test set: {:.2f}'.format(clf.score(X_test, y_test)))

Accuracy of SVM on train set: 1.00
Accuracy of SVM on test set: 0.45


In [54]:
# perceptron
from sklearn.linear_model import perceptron
net = perceptron.Perceptron(n_iter=100, verbose=0, random_state=None, fit_intercept=True, eta0=0.002)
net.fit(X_train, y_train)
y_pred_test = net.predict(X_test)
print('Accuracy of perceptron on train set: {:.2f}'.format(net.score(X_train, y_train)))
print('Accuracy of perceptron on test set: {:.2f}'.format(net.score(X_test, y_test)))

Accuracy of perceptron on train set: 0.54
Accuracy of perceptron on test set: 0.45


/Users/anna/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [55]:
# kNN
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=4)
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(hainan[['BioCNG_cumsum', 'BioCNG_Produced_m3', 'Pig_Manure_t', 'Cassava_t',
       'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t', 'MA', 'Pig_Manure_t**2', 'Cassava_t**2',
       'Fish_waste_water_t**2', 'Kitchen_food_waste_t**2',
       'Municipal_fecal_residue_t**2', 'Tea_waste_t**2', 'Chicken_litter_t**2',
       'Bagasse_feed_t**2', 'Alcohol_waste_t**2',
       'Chinese_medicine_waste_t**2', 'Energy_grass_t**2',
       'Banana_fruit_shafts_t**2', 'Lemon_waste_t**2', 'Percolate_t**2',
       'Other_waste_t**2', 'MA**2']])
hainan_normalized = pd.DataFrame(np_scaled)
hainan_normalized_train, hainan_normalized_test = train_test_split(hainan_normalized, test_size=0.2)
hainan_normalized
Xnor_train = hainan_normalized_train.iloc[:, 1:60]    
ynor_train = hainan_normalized_train[0]
Xnor_test = hainan_normalized_test.iloc[:, 1:60]
ynor_test = hainan_normalized_test[0]
knn.fit(Xnor_train, ynor_train)
y_pred_test = knn.predict(Xnor_test)
print('Accuracy of kNN on train set: {:.2f}'.format(knn.score(Xnor_train, ynor_train)))
print('Accuracy of kNN on test set: {:.2f}'.format(knn.score(Xnor_test, ynor_test)))


Accuracy of kNN on train set: 0.90
Accuracy of kNN on test set: 0.82


In [56]:
# random forest
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators = 1000)
random_forest.fit(X_train, y_train)
print('Accuracy of Random Forest on train set: {:.2f}'.format(random_forest.score(X_train, y_train)))
print('Accuracy of Random Forest on test set: {:.2f}'.format(random_forest.score(X_test, y_test)))

Accuracy of Random Forest on train set: 1.00
Accuracy of Random Forest on test set: 0.43


# Trying with logx

In [61]:
for col in hainan.columns[2:]:
    hainan[col+"log"] = np.log(hainan[col])
hainan.head()


/Users/anna/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


,BioCNG_cumsum,BioCNG_Produced_m3,Pig_Manure_t,Cassava_t,Fish_waste_water_t,Kitchen_food_waste_t,Municipal_fecal_residue_t,Tea_waste_t,Chicken_litter_t,Bagasse_feed_t,...,Chicken_litter_t**2log,Bagasse_feed_t**2log,Alcohol_waste_t**2log,Chinese_medicine_waste_t**2log,Energy_grass_t**2log,Banana_fruit_shafts_t**2log,Lemon_waste_t**2log,Percolate_t**2log,Other_waste_t**2log,MA**2log
0,0,0,6.82,0.0,0.0,0.0,0.0,0.0,0.0,15.0,...,-inf,5.416100,-inf,-inf,-inf,-inf,-inf,-inf,6.549756,15.225662
1,0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,15.0,...,-inf,5.416100,-inf,-inf,-inf,-inf,-inf,-inf,6.729067,15.951129
2,0,0,6.66,0.0,0.0,0.0,0.0,0.0,0.0,15.0,...,-inf,5.416100,-inf,-inf,-inf,-inf,-inf,-inf,6.584253,16.026024
3,0,0,12.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,16.040541
4,0,0,9.54,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,-inf,4.394449,-inf,-inf,-inf,-inf,-inf,-inf,-inf,15.859693


In [62]:
hainan.replace(float('-inf'), 0, inplace = True);
hainan_train, hainan_test = train_test_split(hainan, test_size=0.2)
print('train data len:',len(hainan_train))
print('test data len:',len(hainan_test))
hainan.head()

OverflowError: cannot convert float infinity to integer